# Information Set Decoding (ISD) Attack on Code-Based Cryptosystems

In [1]:
class InformationSetDecoder:
    """
    Implementation of Prange's Information Set Decoding algorithm.
    
    Reference: Prange, E. (1962). "The use of information sets in decoding cyclic codes."
    """
    
    def __init__(self, max_iterations=10000):
        """
        Initialize ISD decoder.
        
        Args:
            max_iterations: Maximum number of random information sets to try
        """
        self.max_iterations = max_iterations
        self.iterations_used = 0
        
    def attack(self, ciphertext, public_key, expected_error_weight):
        """
        Attempt to decode ciphertext using ISD without private key.
        
        Args:
            ciphertext: Received vector c = mG + e
            public_key: Public generator matrix G_hat
            expected_error_weight: Known weight of error vector
            
        Returns:
            (success: bool, recovered_message: vector or None, iterations: int)
        """
        c = ciphertext
        G_hat = public_key
        t = expected_error_weight
        
        k = G_hat.nrows()  # message length
        n = G_hat.ncols()  # codeword length
        
        # Compute parity-check matrix H from G_hat
        # For systematic form [I_k | P], H = [-P^T | I_{n-k}]
        # For general G_hat, use kernel computation
        try:
            H = G_hat.right_kernel().basis_matrix()
        except:
            # Fallback: construct H such that H * G_hat^T = 0
            # This is expensive for large matrices
            return (False, None, 0)
        
        r = H.nrows()  # redundancy = n - k
        
        # Compute syndrome s = H * c^T
        syndrome = H * c
        
        # Prange's algorithm: try random information sets
        for iteration in range(self.max_iterations):
            self.iterations_used = iteration + 1
            
            # Random permutation of column indices
            indices = list(range(n))
            shuffle(indices)
            
            # Split into information set (first k) and redundancy (last r)
            info_set = indices[:k]
            redund_set = indices[k:]
            
            # Permute H and syndrome
            H_perm = H.matrix_from_columns(indices)
            
            # Extract submatrices
            H_info = H_perm.matrix_from_columns(range(k))      # r x k
            H_redund = H_perm.matrix_from_columns(range(k, n)) # r x r
            
            # Check if H_redund is invertible (required for solving)
            if not H_redund.is_invertible():
                continue
            
            # Solve for error in information set positions
            # s = H_info * e_info + H_redund * e_redund
            # Assume e_info = 0 (all errors in redundancy positions)
            # Then: e_redund = H_redund^(-1) * s
            try:
                e_redund_candidate = H_redund.inverse() * syndrome
            except:
                continue
            
            # Check if this gives correct error weight
            if self._weight(e_redund_candidate) != t:
                continue
            
            # Reconstruct full error vector (unpermuted)
            e_candidate = vector(GF(2), n)
            for i, pos in enumerate(redund_set):
                e_candidate[pos] = e_redund_candidate[i]
            
            # Verify: does c - e decode to valid codeword?
            codeword_candidate = c - e_candidate
            
            # Check if it's in the code (H * codeword^T = 0)
            if not (H * codeword_candidate).is_zero():
                continue
            
            # Recover message from codeword
            # Find information set in G_hat
            for attempt in range(100):  # Try multiple info sets for G_hat
                info_positions = sample(range(n), k)
                G_info = G_hat.matrix_from_columns(info_positions)
                
                if G_info.is_invertible():
                    c_info = vector(GF(2), [codeword_candidate[i] for i in info_positions])
                    message = c_info * G_info.inverse()
                    
                    # Verify
                    if message * G_hat == codeword_candidate:
                        return (True, message, self.iterations_used)
                    break
            
            # If we got here with valid error, try systematic recovery
            # This might fail if G_hat is not in systematic form
            
        return (False, None, self.iterations_used)
    
    def _weight(self, vec):
        """Hamming weight of vector."""
        return sum(int(x) for x in vec)


## Attacking McEllience KEM with ISD

In [ ]:
class McElieceKEM_ISDAttacker:
    """
    High-level interface for ISD attacks on McEliece instances.
    Manages attack campaigns and statistics collection.
    """
    
    def __init__(self, max_iterations=10000):
        self.decoder = InformationSetDecoder(max_iterations=max_iterations)
        self.attack_log = []
        
    def attack_mceliece(self, kem_instance, ciphertext, error_weight):
        """
        Attack a McEliece ciphertext.
        
        Args:
            kem_instance: McElieceKEM object (to get public key)
            ciphertext: Encrypted ciphertext
            error_weight: Known error weight used in encryption
            
        Returns:
            dict with attack results
        """
        start_time = time.time()
        
        success, recovered_msg, iterations = self.decoder.attack(
            ciphertext,
            kem_instance.pub_key_G_hat,
            error_weight
        )
        
        attack_time = time.time() - start_time
        
        result = {
            'success': success,
            'iterations': iterations,
            'time': attack_time,
            'n': kem_instance.n,
            'k': kem_instance.k,
            't': error_weight
        }
        
        self.attack_log.append(result)
        return result
    
    def batch_attack(self, r, error_weight, num_trials=20, max_iters=10000):
        """
        Run multiple attacks on fresh McEliece instances.
        
        Args:
            r: Hamming code rank parameter
            error_weight: Error weight to use
            num_trials: Number of independent attack attempts
            max_iters: Max iterations per attack
            
        Returns:
            dict with aggregated statistics
        """
        self.decoder.max_iterations = max_iters
        
        successes = 0
        iterations_list = []
        times_list = []
        
        for trial in range(num_trials):
            # Generate fresh instance
            kem = McElieceKEM(r=r)
            kem.keygen()
            
            # Encrypt with known error weight
            c, _, _ = kem.encapsulation(error_weight=error_weight)
            
            # Attack
            result = self.attack_mceliece(kem, c, error_weight)
            
            if result['success']:
                successes += 1
            iterations_list.append(result['iterations'])
            times_list.append(result['time'])
        
        return {
            'r': r,
            'n': 2**r - 1,
            'k': 2**r - 1 - r,
            't': error_weight,
            'success_rate': float(successes) / float(num_trials),
            'avg_iterations': float(np.mean(iterations_list)),
            'std_iterations': float(np.std(iterations_list)),
            'avg_time': float(np.mean(times_list)),
            'std_time': float(np.std(times_list)),
            'num_trials': num_trials,
            'max_iters': max_iters
        }


# Quick demonstration
print("=" * 70)
print("ISD Attack Demonstration")
print("=" * 70)

# Attack weak parameters
print("\nAttacking McEliece with r=3 (n=7, k=4, t=1)...")
kem_weak = McElieceKEM(r=3)
kem_weak.keygen()
c_weak, k_original, _ = kem_weak.encapsulation(error_weight=1)

attacker = McElieceKEM_ISDAttacker(max_iterations=1000)
result = attacker.attack_mceliece(kem_weak, c_weak, error_weight=1)

print(f"Attack {'SUCCEEDED' if result['success'] else 'FAILED'}")
print(f"Iterations: {result['iterations']}/{attacker.decoder.max_iterations}")
print(f"Time: {result['time']:.4f}s")

# Theoretical success probability
n, k, t = kem_weak.n, kem_weak.k, 1
theoretical_prob = float(binomial(n-k, t)) / float(binomial(n, t))
print(f"\nTheoretical success probability per iteration: {theoretical_prob:.4f}")
print(f"Expected iterations: {1/theoretical_prob:.1f}")

## Attacking BIKE KEM with ISD

In [ ]:
import time
import numpy as np
from sage.all import *


class BikeISDAttacker:
    """
    ISD Attack adapter for BIKE KEM.
    Reuses InformationSetDecoder from Exercise 3.3.
    """
    
    def __init__(self, max_iterations=10000):
        self.decoder = InformationSetDecoder(max_iterations=max_iterations)
        self.attack_log = []
    
    def _build_effective_parity_check(self, pk_h, r):
        """
        Construct effective parity-check matrix for ISD on BIKE.
        
        BIKE: c = e0 + H*e1
        Reformulate as: [I_r | H] * [e0; e1] = c
        
        Returns: H_eff = [I_r | H] (r x 2r matrix)
        """
        # Build circulant matrix H from public key first row
        H_circ = matrix(GF(2), r, r)
        for i in range(r):
            rotated = list(pk_h[-i:]) + list(pk_h[:-i]) if i > 0 else list(pk_h)
            H_circ[i] = vector(GF(2), rotated)
        
        # Effective parity check: [I_r | H]
        I_r = identity_matrix(GF(2), r)
        H_eff = I_r.augment(H_circ)
        
        return H_eff
    
    def attack_bike(self, bike_instance, ciphertext, error_weight, pk=None):
        """
        ISD attack on BIKE ciphertext.
        
        Args:
            bike_instance: BikeKEM instance
            ciphertext: Target ciphertext c
            error_weight: Total error weight t
            pk: Public key (required - first row of H matrix)
        
        Returns:
            dict with attack results
        """
        start_time = time.time()
        
        pk_h = pk
        r = bike_instance.r

        if pk_h is None:
            raise ValueError("Public key 'pk' must be provided to attack_bike")
        
        # Build H_eff = [I_r | H]
        H_eff = self._build_effective_parity_check(pk_h, r)
        
        # For BIKE, the "syndrome" is just the ciphertext
        # We need to find e = [e0 | e1] such that H_eff * e^T = c
        # This is equivalent to standard syndrome decoding
        
        # Pad ciphertext to match generator matrix dimensions if needed
        # Actually, we need to reformulate for the decoder
        # The decoder expects: H * codeword^T = syndrome
        # We have: H_eff * error^T = ciphertext
        
        # Convert parity-check problem H_eff * e^T = syndrome into an equivalent
        # generator-matrix decoding instance that InformationSetDecoder.attack expects.
        # Find a particular solution p to H_eff * x^T = syndrome, then e = p + u
        # where u is any codeword in the nullspace of H_eff. Let G_fake be a generator
        # matrix for the nullspace (rows = basis of right kernel). Then solving
        # p = m*G_fake + e' with wt(e') = t is equivalent to finding e = p - m*G_fake.
        try:
            # Try a direct linear solve for a particular solution
            p = H_eff.solve_right(vector(GF(2), ciphertext))
        except Exception:
            # Fallback: pick an r x r invertible submatrix and solve for its variables
            n = H_eff.ncols()
            indices = list(range(n))
            info_size = n - r
            p = None
            # Try several random permutations to find an invertible redundancy block
            for _ in range(20):
                shuffle(indices)
                try:
                    H_perm = H_eff.matrix_from_columns(indices)
                    H_redund = H_perm.matrix_from_columns(range(info_size, n))
                    if not H_redund.is_invertible():
                        continue
                    p_redund = H_redund.inverse() * vector(GF(2), ciphertext)
                    p = vector(GF(2), [0] * n)
                    redund_set = indices[info_size:]
                    for i, pos in enumerate(redund_set):
                        p[pos] = p_redund[i]
                    break
                except Exception:
                    continue
            if p is None:
                raise RuntimeError("Could not find particular solution for H_eff * x = syndrome")

        # Build generator matrix for nullspace (rows = basis vectors)
        kernel = H_eff.right_kernel()
        basis = kernel.basis()
        if len(basis) == 0:
            raise RuntimeError("Right kernel is empty; cannot form generator matrix")
        G_rows = [vector(GF(2), b) for b in basis]
        G_fake = matrix(GF(2), G_rows)

        # Use the existing InformationSetDecoder on (p, G_fake)
        success, recovered_msg, iterations = self.decoder.attack(
            p,
            G_fake,
            error_weight
        )

        recovered_error = None
        if success and recovered_msg is not None:
            # recovered_msg is the message m (length k); compute codeword and error
            codeword = recovered_msg * G_fake
            recovered_error = p - codeword
        elif success and recovered_msg is None:
            # Some decoders may return the error directly in place of message; try to treat p as codeword
            recovered_error = p
        else:
            recovered_error = None
        
        attack_time = time.time() - start_time
        
        result = {
            'success': success,
            'iterations': iterations,
            'time': attack_time,
            'r': r,
            'w': bike_instance.w,
            't': error_weight
        }
        
        self.attack_log.append(result)
        return result
    
    def batch_attack(self, r, w, t, num_trials=15, max_iters=10000):
        """
        Run multiple ISD attacks on fresh BIKE instances.
        
        Args:
            r: BIKE block size (must be prime)
            w: BIKE row weight  
            t: Error weight
            num_trials: Number of independent attacks
            max_iters: Max iterations per attack
        
        Returns:
            dict with aggregated statistics
        """
        self.decoder.max_iterations = max_iters
        
        successes = 0
        iterations_list = []
        times_list = []
        
        for trial in range(num_trials):
            # Generate fresh BIKE instance using original BikeKEM from Cell 7
            bike = BikeKEM(r=r, w=w, t=t)
            sk, pk = bike.key_gen()
            
            # Encapsulate
            c, (K, (e0, e1)) = bike.encaps(pk)

            # Perform attack; catch per-trial exceptions so sweep continues
            try:
                result = self.attack_bike(bike, c, t, pk=pk)
            except Exception as exc:
                # Log (optional) and treat as failed trial
                # print(f"Trial error r={r} w={w} t={t}: {exc}")
                result = {
                    'success': False,
                    'iterations': 0,
                    'time': float('inf')
                }

            if result.get('success'):
                successes += 1
            iterations_list.append(result.get('iterations', self.decoder.max_iterations))
            times_list.append(result.get('time', float('inf')))
        
        return {
            'r': r,
            'w': w,
            't': t,
            'success_rate': float(successes) / float(num_trials),
            'avg_iterations': float(np.mean(iterations_list)),
            'std_iterations': float(np.std(iterations_list)),
            'avg_time': float(np.mean(times_list)),
            'std_time': float(np.std(times_list)),
            'num_trials': num_trials,
            'max_iters': max_iters
        }


# ============================================================
# Demonstration: Attack vulnerable BIKE parameters
# ============================================================

print("=" * 70)
print("ISD Attack on BIKE KEM - Demonstration")
print("=" * 70)

print("\nUsing BikeKEM class from Cell 7 (Exercise 3.1)")
print("-" * 50)

print("\nAttacking BIKE with r=53, w=6, t=2 (very vulnerable)...")
bike_weak = BikeKEM(r=53, w=6, t=2)
sk, pk = bike_weak.key_gen()

c_weak, (K_orig, (e0_orig, e1_orig)) = bike_weak.encaps(pk)

attacker = BikeISDAttacker(max_iterations=5000)
result = attacker.attack_bike(bike_weak, c_weak, error_weight=2, pk=pk)

print(f"Iterations: {result['iterations']}/{attacker.decoder.max_iterations}")
print(f"Time: {result['time']:.4f}s")

# Theoretical expectation
n_total = 2 * bike_weak.r
r_space = bike_weak.r
t_val = 2
theoretical_prob = float(binomial(r_space, t_val)) / float(binomial(n_total, t_val))
print(f"\nApproximate success probability per iteration: {theoretical_prob:.6f}")
print(f"Expected iterations: {1/theoretical_prob if theoretical_prob > 0 else float('inf'):.1f}")